In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Sales df
path = "data\sales_with_dates.csv"

sales_df = pd.read_csv(path, sep=",", encoding="utf-8")

sales_df.head(3)

,id,item,category,department,store_code,day,daily_sales,date,weekday,weekday_int,event
0,ACCESORIES_1_001_NYC_1,1,ACCESORIES,1,NYC_1,d_1,0,2011-01-29,Saturday,1,none
1,ACCESORIES_1_002_NYC_1,2,ACCESORIES,1,NYC_1,d_1,0,2011-01-29,Saturday,1,none
2,ACCESORIES_1_003_NYC_1,3,ACCESORIES,1,NYC_1,d_1,0,2011-01-29,Saturday,1,none


In [5]:
sales_df = sales_df.drop(columns=['item','category','department','store_code'])

In [15]:
sales_df.head(3)

,id,day,daily_sales,date,weekday,weekday_int,event
0,ACCESORIES_1_001_NYC_1,d_1,0,2011-01-29,Saturday,1,none
1,ACCESORIES_1_002_NYC_1,d_1,0,2011-01-29,Saturday,1,none
2,ACCESORIES_1_003_NYC_1,d_1,0,2011-01-29,Saturday,1,none


In [22]:
sales_df['day'].nunique(), sales_df['id'].nunique()

(1913, 30490)

In [18]:
sales_df.shape

(58327370, 7)

In [7]:
sales_df['date'] = pd.to_datetime(sales_df['date'])

In [8]:
# Extract yearweek
sales_df['yearweek'] = sales_df['date'].dt.strftime('%G%V')

In [9]:
sales_df = sales_df.drop(columns=['weekday','weekday_int'])

In [10]:
sales_df.head(3)

,id,day,daily_sales,date,event,yearweek
0,ACCESORIES_1_001_NYC_1,d_1,0,2011-01-29,none,201104
1,ACCESORIES_1_002_NYC_1,d_1,0,2011-01-29,none,201104
2,ACCESORIES_1_003_NYC_1,d_1,0,2011-01-29,none,201104


In [11]:
# sales_df.to_csv('data\sales_clean.csv', index=False)

In [34]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           object        
 1   day          object        
 2   daily_sales  int64         
 3   date         datetime64[ns]
 4   event        object        
 5   yearweek     object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 2.6+ GB


In [33]:
# Prices df
path = "data\original\item_prices.csv"

prices_df = pd.read_csv(path, sep=",", encoding="utf-8")

prices_df.head(3)

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.00,12.74
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.00,12.74
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.00,10.99


In [35]:
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [40]:
prices_df[prices_df['yearweek'].isna()].head(3)

,item,category,store_code,yearweek,sell_price
149,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.15
150,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.15
151,ACCESORIES_1_001,ACCESORIES,NYC_1,NaN,11.15


In [42]:
prices_df = prices_df[prices_df['yearweek'].notna()]

In [43]:
prices_df.loc[:,'yearweek'] = prices_df.loc[:,'yearweek'].astype(int).astype(str)

C:\Users\alexa\AppData\Local\Temp\ipykernel_11036\2023360159.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['201328' '201329' '201330' ... '201615' '201616' '201617']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  prices_df.loc[:,'yearweek'] = prices_df.loc[:,'yearweek'].astype(int).astype(str)


In [44]:
prices_df.head(3)

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.74
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329,12.74
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330,10.99


In [45]:
# Crear columna id -> item + store_code -> para poder hacer merges y llevar el precio al ds principal
prices_df['id'] = prices_df['item'] + '_' + prices_df['store_code']

In [46]:
prices_df = prices_df.drop(columns=['item','store_code','category',])

In [47]:
prices_df.head(3)

,yearweek,sell_price,id
0,201328,12.74,ACCESORIES_1_001_NYC_1
1,201329,12.74,ACCESORIES_1_001_NYC_1
2,201330,10.99,ACCESORIES_1_001_NYC_1


In [50]:
prices_df.shape

(6721786, 3)

In [ ]:
# Sales clean
sales_df = pd.read_csv('data\sales_clean.csv', sep=",", encoding="utf-8")

In [14]:
# Prices clean
prices_df = pd.read_csv('data\prices_clean.csv', sep=",", encoding="utf-8")

In [18]:
prices_df['yearweek'] = prices_df['yearweek'].astype(str)

In [27]:
# Fusionar los DataFrames en base a las columnas 'id' y 'yearweek'
merged_df = sales_df.merge(prices_df[['id', 'yearweek', 'sell_price']], on=['id', 'yearweek'], how='left')

In [28]:
merged_df.head(3)

,id,day,daily_sales,date,event,yearweek,sell_price
0,ACCESORIES_1_001_NYC_1,d_1,0,2011-01-29,none,201104,NaN
1,ACCESORIES_1_002_NYC_1,d_1,0,2011-01-29,none,201104,NaN
2,ACCESORIES_1_003_NYC_1,d_1,0,2011-01-29,none,201104,NaN


In [29]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           object        
 1   day          object        
 2   daily_sales  int64         
 3   date         datetime64[ns]
 4   event        object        
 5   yearweek     object        
 6   sell_price   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 3.0+ GB


In [30]:
merged_df['sell_price'] = merged_df['sell_price'].fillna(0)

In [31]:
merged_df.head(3)

,id,day,daily_sales,date,event,yearweek,sell_price
0,ACCESORIES_1_001_NYC_1,d_1,0,2011-01-29,none,201104,0.00
1,ACCESORIES_1_002_NYC_1,d_1,0,2011-01-29,none,201104,0.00
2,ACCESORIES_1_003_NYC_1,d_1,0,2011-01-29,none,201104,0.00


In [32]:
merged_df.isna().sum()

id             0
day            0
daily_sales    0
date           0
event          0
yearweek       0
sell_price     0
dtype: int64

In [33]:
merged_df.duplicated().sum()

0

In [34]:
# Guardamos para no tener que volver a hacer esto
# merged_df.to_csv('data\sales_prices_clean.csv', index=False)

In [36]:
df = merged_df.copy()

In [41]:
df.head(3)

,id,day,daily_sales,date,event,yearweek,sell_price
0,ACCESORIES_1_001_NYC_1,d_1,0,2011-01-29,none,201104,0.00
1,ACCESORIES_1_002_NYC_1,d_1,0,2011-01-29,none,201104,0.00
2,ACCESORIES_1_003_NYC_1,d_1,0,2011-01-29,none,201104,0.00


In [37]:
# Agrupar por año y producto, sumar las ventas diarias y encontrar el producto más vendido en cada año
popular_products = df.groupby([df['date'].dt.year, 'id'])['daily_sales'].sum().reset_index()
popular_products = popular_products.sort_values(by=['date', 'daily_sales'], ascending=[True, False]).groupby('date').head(10)


In [39]:
# Productos populares por año segun el total de daily sales
popular_products.head(10)

,date,id,daily_sales
28101,2011,SUPERMARKET_3_586_BOS_2,36574
28102,2011,SUPERMARKET_3_586_BOS_3,34053
23145,2011,SUPERMARKET_3_090_NYC_3,28466
28115,2011,SUPERMARKET_3_587_NYC_3,25464
28100,2011,SUPERMARKET_3_586_BOS_1,23506
28105,2011,SUPERMARKET_3_586_NYC_3,22785
28109,2011,SUPERMARKET_3_586_PHI_3,21474
27791,2011,SUPERMARKET_3_555_BOS_2,20968
27792,2011,SUPERMARKET_3_555_BOS_3,20777
24761,2011,SUPERMARKET_3_252_BOS_2,20133


In [47]:
# Productos populares 2015 segun el total de daily sales
popular_products[popular_products['date']==2015].head(10)

,date,id,daily_sales
150061,2015,SUPERMARKET_3_586_BOS_2,33811
145105,2015,SUPERMARKET_3_090_NYC_3,33279
145405,2015,SUPERMARKET_3_120_NYC_3,31598
150062,2015,SUPERMARKET_3_586_BOS_3,27030
150065,2015,SUPERMARKET_3_586_NYC_3,26792
146721,2015,SUPERMARKET_3_252_BOS_2,21479
145109,2015,SUPERMARKET_3_090_PHI_3,19883
145102,2015,SUPERMARKET_3_090_BOS_3,19415
145103,2015,SUPERMARKET_3_090_NYC_1,18482
150060,2015,SUPERMARKET_3_586_BOS_1,18387


In [42]:
# Agrupar por producto y sumar las ventas diarias
popular_products_total = df.groupby('id')['daily_sales'].sum().reset_index()

# Ordenar los productos por el número total de ventas en orden descendente
popular_products_total = popular_products_total.sort_values(by='daily_sales', ascending=False)

In [45]:
# Productos populares segun el total de daily sales -> Falta quitar las tiendas para ver el total de verdad
popular_products_total.head(10)

,id,daily_sales
23145,SUPERMARKET_3_090_NYC_3,250502
28101,SUPERMARKET_3_586_BOS_2,192835
28102,SUPERMARKET_3_586_BOS_3,150122
28105,SUPERMARKET_3_586_NYC_3,134386
23143,SUPERMARKET_3_090_NYC_1,127203
23149,SUPERMARKET_3_090_PHI_3,121434
23141,SUPERMARKET_3_090_BOS_2,119496
23142,SUPERMARKET_3_090_BOS_3,114854
24761,SUPERMARKET_3_252_BOS_2,114153
28100,SUPERMARKET_3_586_BOS_1,112454


In [46]:
# Productos menos populares historicamente segun el total de daily sales -> por tienda!!!
popular_products_total.tail(10)

,id,daily_sales
14026,HOME_&_GARDEN_2_307_NYC_4,21
10676,HOME_&_GARDEN_1_512_NYC_4,20
6355,HOME_&_GARDEN_1_073_NYC_3,19
13116,HOME_&_GARDEN_2_216_NYC_4,19
8945,HOME_&_GARDEN_1_336_NYC_3,18
18848,SUPERMARKET_2_057_PHI_2,16
18982,SUPERMARKET_2_071_BOS_3,16
1659,ACCESORIES_1_170_PHI_3,13
29994,SUPERMARKET_3_778_NYC_2,12
5845,HOME_&_GARDEN_1_020_NYC_3,10


In [48]:
popular_products_total['item'] = popular_products_total['id'].str.rsplit('_', n=2).str[0]

In [49]:
popular_products_total.head(3)

,id,daily_sales,item
23145,SUPERMARKET_3_090_NYC_3,250502,SUPERMARKET_3_090
28101,SUPERMARKET_3_586_BOS_2,192835,SUPERMARKET_3_586
28102,SUPERMARKET_3_586_BOS_3,150122,SUPERMARKET_3_586


In [51]:
# Popular products global
popular_items_total = popular_products_total.groupby('item')['daily_sales'].sum().reset_index()

In [53]:
popular_items_total = popular_items_total.sort_values('daily_sales',ascending=False)

In [54]:
# Mas populasres Global
popular_items_total.head(10)

,item,daily_sales
2314,SUPERMARKET_3_090,1002529
2810,SUPERMARKET_3_586,920242
2476,SUPERMARKET_3_252,565299
2779,SUPERMARKET_3_555,491287
2937,SUPERMARKET_3_714,396172
2811,SUPERMARKET_3_587,396119
2918,SUPERMARKET_3_694,390001
2450,SUPERMARKET_3_226,363082
2426,SUPERMARKET_3_202,295689
2946,SUPERMARKET_3_723,284333


In [55]:
# Menos populares Global
popular_items_total.tail(10)

,item,daily_sales
438,ACCESORIES_2_023,800
1402,HOME_&_GARDEN_2_307,796
1226,HOME_&_GARDEN_2_130,789
1340,HOME_&_GARDEN_2_245,780
526,ACCESORIES_2_111,770
1101,HOME_&_GARDEN_2_005,757
499,ACCESORIES_2_084,746
1271,HOME_&_GARDEN_2_175,718
534,ACCESORIES_2_119,660
1197,HOME_&_GARDEN_2_101,568


In [56]:
popular_items_total.describe()

,daily_sales
count,3049.00
mean,21546.54
std,44252.43
min,568.00
25%,4310.00
50%,9526.00
75%,21406.00
max,1002529.00
